# Main Script

STATIC VERSION
- Given a fixed DoD
- Main function
- Auxillary functions:
    - initial solution creator: fill up vehicles while respecting capacity
    - (greedy) insertion heuristic: in stead of looking for all possible insertions, look for a better insertion until you find no better one
        - initial solution creator > spatio-temporal clustering of requests?
        - feasibility check
        - incremental waiting time + travel time calculator
    - improvement heuristic
        - neighborhood creators (destroy&repair, swappers, L-opt moves, ...)
        - evaluate objective functions

## Todo (update 4/2)

- Test basic network loading functions for larger instances!
- Let non-taken requests 'spill over' into a new request group > create functions
- In the current initial solution, (1,3) travellers benefit a lot <> other travellers: so maybe put a limit on how many travellers can be taken by those
- Some vehicles can drive all the way to 5 (1 > 5 > 1), others return from 3 directly back (1 > 3 > 1)
    - make it that some vehicles are available at the depot: these can be schedules for new rides
- Create travel time evaluators

In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

import od_traveltime_generation as od
import requests_generation as rg
import solution_generation as sg
import traveltime_evaluation as te
import solution_visualisation as sv

In [2]:
# PARAMETERS

#network
network_size = 'small'
interstop_distance = 'half'
v_mean = 50 #km/h

#requests
demand_scenario = 2
time_of_day = 1 #1 = peak, 0 = off-peak
peak_duration = 60 #min.
req_max_cluster_time = 5 #min.

#vehicles
available_vehicles = 30
vehicle_capacity = 20
dod = 0   #static version = 0, dynamic version > 0

#objective function
WT_pen = 3 
TT_pen = 1 

## Load network

In [3]:
# network
network = od.import_network(network_size, interstop_distance)
od_matrix = od.generate_tt_od(network, v_mean)
od_matrix

{(1, 1): 0.0,
 (1, 2): 5.9687184554140265,
 (1, 3): 6.0,
 (1, 4): 5.9687184554140265,
 (1, 5): 0.0,
 (2, 1): 5.9687184554140265,
 (2, 2): 0.0,
 (2, 3): 5.9687184554140265,
 (2, 4): 0.0,
 (2, 5): 5.9687184554140265,
 (3, 1): 6.0,
 (3, 2): 5.9687184554140265,
 (3, 3): 0.0,
 (3, 4): 5.9687184554140265,
 (3, 5): 6.0,
 (4, 1): 5.9687184554140265,
 (4, 2): 0.0,
 (4, 3): 5.9687184554140265,
 (4, 4): 0.0,
 (4, 5): 5.9687184554140265,
 (5, 1): 0.0,
 (5, 2): 5.9687184554140265,
 (5, 3): 6.0,
 (5, 4): 5.9687184554140265,
 (5, 5): 0.0}

## Import requests

In [4]:
lambdapeak = rg.get_scenario_mean_demand('city', network_size, scen=demand_scenario, peak=1)
mupeak = rg.get_scenario_mean_demand('terminal', network_size, scen=demand_scenario, peak=1)

In [5]:
dict_requests = rg.convert_md_todict(lambdapeak, mupeak, demand_scenario)
total_requests = rg.generate_requests(dict_requests, peak_duration)
od_pairs = set(total_requests.keys())

In [6]:
list_all_requests = rg.list_all_requests(total_requests)
list_all_requests

[((1, 2), 0, 0),
 ((1, 2), 7.9587450816311005, 0),
 ((1, 2), 15.917490163262201, 0),
 ((1, 2), 23.8762352448933, 0),
 ((1, 2), 31.834980326524402, 0),
 ((1, 2), 39.7937254081555, 0),
 ((1, 2), 47.7524704897866, 0),
 ((1, 2), 55.7112155714177, 0),
 ((1, 3), 0, 0),
 ((1, 3), 0.1989686270407775, 0),
 ((1, 3), 0.397937254081555, 0),
 ((1, 3), 0.5969058811223324, 0),
 ((1, 3), 0.79587450816311, 0),
 ((1, 3), 0.9948431352038876, 0),
 ((1, 3), 1.1938117622446651, 0),
 ((1, 3), 1.3927803892854427, 0),
 ((1, 3), 1.5917490163262202, 0),
 ((1, 3), 1.7907176433669978, 0),
 ((1, 3), 1.9896862704077753, 0),
 ((1, 3), 2.188654897448553, 0),
 ((1, 3), 2.3876235244893302, 0),
 ((1, 3), 2.5865921515301076, 0),
 ((1, 3), 2.785560778570885, 0),
 ((1, 3), 2.9845294056116622, 0),
 ((1, 3), 3.1834980326524396, 0),
 ((1, 3), 3.382466659693217, 0),
 ((1, 3), 3.5814352867339942, 0),
 ((1, 3), 3.7804039137747716, 0),
 ((1, 3), 3.979372540815549, 0),
 ((1, 3), 4.178341167856327, 0),
 ((1, 3), 4.377309794897104, 0

In [7]:
# requests are grouped per 5 minutes from the 1st departure of a group
copy_list_requests = list_all_requests.copy()
grouped_requests = rg.group_requests_dt(copy_list_requests, req_max_cluster_time, od_pairs)
grouped_requests

{(2,
  3): [[((2, 3), 0, 0),
   ((2, 3), 0.1989686270407775, 0),
   ((2, 3), 0.397937254081555, 0),
   ((2, 3), 0.5969058811223324, 0),
   ((2, 3), 0.79587450816311, 0),
   ((2, 3), 0.9948431352038876, 0),
   ((2, 3), 1.1938117622446651, 0),
   ((2, 3), 1.3927803892854427, 0),
   ((2, 3), 1.5917490163262202, 0),
   ((2, 3), 1.7907176433669978, 0),
   ((2, 3), 1.9896862704077753, 0),
   ((2, 3), 2.188654897448553, 0),
   ((2, 3), 2.3876235244893302, 0),
   ((2, 3), 2.5865921515301076, 0),
   ((2, 3), 2.785560778570885, 0),
   ((2, 3), 2.9845294056116622, 0),
   ((2, 3), 3.1834980326524396, 0),
   ((2, 3), 3.382466659693217, 0),
   ((2, 3), 3.5814352867339942, 0),
   ((2, 3), 3.7804039137747716, 0),
   ((2, 3), 3.979372540815549, 0),
   ((2, 3), 4.178341167856327, 0),
   ((2, 3), 4.377309794897104, 0),
   ((2, 3), 4.576278421937881, 0),
   ((2, 3), 4.775247048978659, 0),
   ((2, 3), 4.974215676019436, 0)], [((2, 3), 5.173184303060213, 0),
   ((2, 3), 5.372152930100991, 0),
   ((2, 3), 5.

In [8]:
# this shows how big the groups of passengers are (per threshold time from the first departure) + how many there are
count_groups = rg.request_groups_per_od(grouped_requests)
count_groups

{(2, 3): [26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 16],
 (1, 3): [26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 16],
 (4, 5): [1, 1],
 (1, 2): [1, 1, 1, 1, 1, 1, 1, 1],
 (3, 5): [1, 1, 1, 1, 1, 1, 1, 1],
 (3, 4): [1, 1, 1, 1, 1, 1, 1, 1]}

## Initial solution generation

Insertion strategy: momentarily focussed on the capacity constraint (i.e. trying to fill vehicles), not so much on the time constraint

1. Divide requests into chunks: 
 * (a) of at most equal size (e.g. 10) 
 * (b) group requests as long as the maximum waiting time is e.g. 5 min.
2. Select a first chunk of travellers, who want to travel from the origin to the farthest point, and fill a bus with them (almost) 
    * If this bus is ride is full, then this schedule is complete.
    * If there is room left, then add passengers in between at each visited stop.


In [9]:
tocity_keys = [(1,2),(2,3),(1,3)]
toterminal_keys = [(3,4),(3,5),(4,5)]

filterByKey = lambda keys: {x: grouped_requests[x] for x in keys}
tocity_requests = filterByKey(tocity_keys)
toterminal_requests = filterByKey(toterminal_keys)

tocity_requests

{(1, 2): [[((1, 2), 0, 0)],
  [((1, 2), 7.9587450816311005, 0)],
  [((1, 2), 15.917490163262201, 0)],
  [((1, 2), 23.8762352448933, 0)],
  [((1, 2), 31.834980326524402, 0)],
  [((1, 2), 39.7937254081555, 0)],
  [((1, 2), 47.7524704897866, 0)],
  [((1, 2), 55.7112155714177, 0)]],
 (2,
  3): [[((2, 3), 0, 0),
   ((2, 3), 0.1989686270407775, 0),
   ((2, 3), 0.397937254081555, 0),
   ((2, 3), 0.5969058811223324, 0),
   ((2, 3), 0.79587450816311, 0),
   ((2, 3), 0.9948431352038876, 0),
   ((2, 3), 1.1938117622446651, 0),
   ((2, 3), 1.3927803892854427, 0),
   ((2, 3), 1.5917490163262202, 0),
   ((2, 3), 1.7907176433669978, 0),
   ((2, 3), 1.9896862704077753, 0),
   ((2, 3), 2.188654897448553, 0),
   ((2, 3), 2.3876235244893302, 0),
   ((2, 3), 2.5865921515301076, 0),
   ((2, 3), 2.785560778570885, 0),
   ((2, 3), 2.9845294056116622, 0),
   ((2, 3), 3.1834980326524396, 0),
   ((2, 3), 3.382466659693217, 0),
   ((2, 3), 3.5814352867339942, 0),
   ((2, 3), 3.7804039137747716, 0),
   ((2, 3), 3

In [10]:
first_stop = 1
last_stop = 3
first_veh_index = 1

copy_tocity_requests = tocity_requests.copy()

In [11]:
initial = sg.create_initial_solution(grouped_requests, first_stop, last_stop, first_veh_index, available_vehicles, vehicle_capacity)

In [15]:
corr_initial = sg.correct_dep_times(initial, od_matrix)

## Evaluate travel time

In [16]:
waiting_time_dict = te.calc_waiting_time(corr_initial)
inveh_time_dict = te.calc_in_vehicle_time(corr_initial, od_matrix)
total_tt_dict = te.calculate_ttt(inveh_time_dict, waiting_time_dict)

In [17]:
sum_stops = te.sum_total_tt(total_tt_dict, level='stop')
sum_vehicle = te.sum_total_tt(total_tt_dict, level='vehicle')
sum_total = te.sum_total_tt(total_tt_dict, level='total')

In [18]:
sum_stops

{1: {1: 136.08739342389654, 2: 128.3279573251155, 3: 0},
 2: {1: 176.08008745909282, 2: 170.1113690036788, 3: 0},
 3: {1: 146.23751624945058, 2: 140.26879779403657, 3: 0},
 4: {1: 303.4200087651904, 2: 297.4512903097764, 3: 0},
 5: {1: 363.1105968774237, 2: 357.14187842200965, 3: 0},
 6: {1: 258.4558219004491, 2: 252.4871034450351, 3: 0},
 7: {1: 490.4505181835214, 2: 484.4817997281074, 3: 0},
 8: {1: 550.1411062957546, 2: 544.1723878403405, 3: 0},
 9: {1: 74.60915087057998, 2: 162.9457755837364, 3: 0},
 10: {1: 128.3279573251155, 2: 112.41046716185333, 3: 0},
 11: {1: 128.3279573251155, 2: 112.41046716185333, 3: 0},
 12: {1: 74.60915087057998, 2: 140.66128935516934, 3: 0},
 13: {1: 128.3279573251155, 2: 96.49297699859115, 3: 0},
 14: {1: 128.3279573251155, 2: 96.49297699859115, 3: 0},
 15: {1: 74.60915087057998, 2: 118.3768031266023, 3: 0},
 16: {1: 128.3279573251155, 2: 80.5754868353289, 3: 0},
 17: {1: 128.3279573251155, 2: 80.57548683532889, 3: 0},
 18: {1: 74.60915087057998, 2: 97

In [19]:
sum_vehicle

{1: 264.4153507490121,
 2: 346.1914564627716,
 3: 286.5063140434871,
 4: 600.8712990749668,
 5: 720.2524752994334,
 6: 510.94292534548424,
 7: 974.9323179116288,
 8: 1094.3134941360952,
 9: 237.5549264543164,
 10: 240.73842448696882,
 11: 240.73842448696882,
 12: 215.27044022574933,
 13: 224.82093432370664,
 14: 224.82093432370664,
 15: 192.98595399718226,
 16: 208.9034441604444,
 17: 208.9034441604444,
 18: 171.8973216657156,
 19: 194.1818078942827,
 20: 194.1818078942827,
 21: 170.31101834233812,
 22: 198.96181992459145,
 23: 198.96181992459145,
 24: 190.20992318127162,
 25: 214.87931008785364,
 26: 214.87931008785364,
 27: 212.49440940983868,
 28: 230.79680025111583,
 29: 147.22997689398935,
 30: 38.79684013809582}

In [20]:
sum_total

9170.944725338188

## Solution visualisation

In [22]:
df = sv.convert_to_dataframe(corr_initial)
df.head()

,0,1,2,3
"(1, 1)",1.790718,"[((1, 3), 0, 0), ((1, 3), 0.1989686270407775, ...","[((1, 2), 0, 0)]",None
"(1, 2)",7.759436,"[((1, 3), 0, 0), ((1, 3), 0.1989686270407775, ...","[((2, 3), 0, 0), ((2, 3), 0.1989686270407775, ...",None
"(1, 3)",NaN,None,None,None
"(2, 1)",7.958745,"[((1, 3), 1.9896862704077753, 0), ((1, 3), 2.1...","[((1, 2), 7.9587450816311005, 0)]",None
"(2, 2)",13.927464,"[((1, 3), 1.9896862704077753, 0), ((1, 3), 2.1...","[((2, 3), 1.9896862704077753, 0), ((2, 3), 2.1...",None
